In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pylab as plt

import sys
sys.path.append('..')

import tidy3d as td
import tidy3d.web as web

In [2]:
# set up parameters of simulation
dl=0.01
pml = td.PMLLayer(profile='standard', num_layers=10)
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 10/fwidth

# create structure
dielectric = td.nk_to_medium(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]),
    medium=dielectric)

# create source
source = td.VolumeSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time = td.GaussianPulse(
        freq0=freq0,
        fwidth=fwidth),
    polarization='Jx')

# create monitor
monitor = td.FieldMonitor(
    fields=['Ex', 'Hy'],
    center=(0, 0, 0),
    size=(4, 4, 0),
    freqs=[freq0])

# Initialize simulation
sim = td.Simulation(size=sim_size,
                    grid_size=(dl, dl, dl),
                    structures=[square],
                    sources={'source': source},
                    monitors={'monitor': monitor},
                    run_time=run_time,
                    pml_layers=(pml, pml, pml))

In [ ]:
task_id = web.upload(sim, task_name='quickstart')
web.start(task_id)
web.monitor(task_id)

[autoreload of tidy3d.components.simulation failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
  File "pydantic/main.py", line 422, in pydantic.main.BaseModel.__setattr__
ValueError: "Simulation" object has no field "__class__"
]


[09:25:58] INFO     Creating task.                                              ]8;id=127482;file:///Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/notebooks/../tidy3d/web/webapi.py\webapi.py]8;;\:329

[09:25:59] INFO     Uploading the json file                                     ]8;id=519421;file:///Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/notebooks/../tidy3d/web/webapi.py\webapi.py]8;;\:338

Output()

[09:26:00] status = queued                                                      ]8;id=477164;file:///Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/tidy3d/web/webapi.py\webapi.py]8;;\:157

[09:26:09] status = preprocess                                                  ]8;id=772608;file:///Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/tidy3d/web/webapi.py\webapi.py]8;;\:157

[09:26:32] status = running                                                     ]8;id=273432;file:///Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/tidy3d/web/webapi.py\webapi.py]8;;\:157

In [ ]:
web.download(task_id, simulation=sim, path='data/sim_data.hdf5')

In [ ]:
sim_data = web.load_data(task_id, simulation=sim, path='data/sim_data.hdf5')
ax = sim_data['monitor'].Ex.isel(f=0, z=0).imag.plot.pcolormesh(x='x', y='y', vmin=-5e-13, vmax=5e-13, cmap='RdBu')

plt.show()

In [ ]:
sim_data.log